# Feature Engineering - Silver to Silver_ML

Ce notebook transforme les données Silver en features ML-ready avec :
- Nettoyage supplémentaire
- Features temporelles avec Window Functions
- Features de fenêtre glissante (rolling)
- Label de classification (flight_phase)

In [ ]:
# Cellule 1 : Configuration
%pip install python-dotenv

import os
from dotenv import load_dotenv
from pyspark.sql import SparkSession

# Charger les variables d'environnement depuis le fichier .env
load_dotenv()

GARAGE_ENDPOINT = os.getenv("GARAGE_ENDPOINT", "http://garage:3900")
ACCESS_KEY = os.getenv("ACCESS_KEY")
SECRET_KEY = os.getenv("SECRET_KEY")
BUCKET_NAME = os.getenv("BUCKET_NAME", "datalake")

if not ACCESS_KEY or not SECRET_KEY:
    raise ValueError("❌ ACCESS_KEY et SECRET_KEY doivent être définis dans le fichier .env")

# Définir les chemins
SILVER_PATH = f"s3a://{BUCKET_NAME}/silver/flights"
SILVER_ML_PATH = f"s3a://{BUCKET_NAME}/silver/flights_ml"

print("✅ Configuration chargée depuis .env")
print(f"📂 Silver Path: {SILVER_PATH}")
print(f"📂 Silver ML Path: {SILVER_ML_PATH}")

In [ ]:
# Initialisation de Spark avec configuration S3/Delta

# 1. Packages
packages = [
    "org.apache.hadoop:hadoop-aws:3.3.4",
    "com.amazonaws:aws-java-sdk-bundle:1.12.262",
    "org.apache.spark:spark-hadoop-cloud_2.12:3.5.3",
    "io.delta:delta-spark_2.12:3.0.0"
]

# 2. Configuration Spark
spark = SparkSession.builder \
    .appName("FeatureEngineering_SilverML") \
    .config("spark.jars.packages", ",".join(packages)) \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.endpoint", GARAGE_ENDPOINT) \
    .config("spark.hadoop.fs.s3a.access.key", ACCESS_KEY) \
    .config("spark.hadoop.fs.s3a.secret.key", SECRET_KEY) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.hadoop.fs.s3a.endpoint.region", "garage") \
    .config("spark.hadoop.fs.s3a.committer.name", "filesystem") \
    .config("spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", "2") \
    .config("spark.hadoop.fs.s3a.multiobjectdelete.enable", "false") \
    .config("spark.sql.shuffle.partitions", "10") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")
print("✅ Spark Session configurée pour Feature Engineering")

In [ ]:
# Cellule 2 : Lecture des données Silver

print(f"📖 Lecture de la table Delta Silver depuis {SILVER_PATH}...")
df_silver = spark.read.format("delta").load(SILVER_PATH)

# Afficher le schéma
print("\n📋 Schéma des données Silver :")
df_silver.printSchema()

# Afficher le nombre de lignes
count = df_silver.count()
print(f"\n📊 Nombre total de lignes : {count:,}")

# Afficher un aperçu
print("\n🔍 Aperçu des données :")
df_silver.show(5, truncate=False)

In [ ]:
# Cellule 3 : Nettoyage supplémentaire

from pyspark.sql.functions import col

print("🧹 Nettoyage des données...")

# Compter avant nettoyage
count_before = df_silver.count()
print(f"Lignes avant nettoyage : {count_before:,}")

# Filtrer les données invalides
df_cleaned = df_silver \
    .filter(col("icao24").isNotNull()) \
    .filter(col("altitude_meters").between(-500, 15000)) \
    .filter(col("velocity_kmh").between(0, 1200))

# Compter après nettoyage
count_after = df_cleaned.count()
removed = count_before - count_after
print(f"Lignes après nettoyage : {count_after:,}")
print(f"Lignes supprimées : {removed:,} ({100 * removed / count_before:.2f}%)")

print("\n✅ Nettoyage terminé")

In [ ]:
# Cellule 4 : Features temporelles avec Window Functions

from pyspark.sql.window import Window
from pyspark.sql.functions import lag, lead, avg, stddev, col, when, row_number

print("🪟 Création des features temporelles avec Window Functions...")

# Fenêtre par avion, ordonnée par timestamp
window_aircraft = Window.partitionBy("icao24").orderBy("event_timestamp")

# Créer les features temporelles
df_temporal = df_cleaned \
    .withColumn("prev_altitude", lag("altitude_meters", 1).over(window_aircraft)) \
    .withColumn("prev_velocity", lag("velocity_kmh", 1).over(window_aircraft)) \
    .withColumn("altitude_change", col("altitude_meters") - col("prev_altitude")) \
    .withColumn("velocity_change", col("velocity_kmh") - col("prev_velocity")) \
    .withColumn("observation_rank", row_number().over(window_aircraft))

print("✅ Features temporelles créées :")
print("   - prev_altitude: Altitude précédente")
print("   - prev_velocity: Vitesse précédente")
print("   - altitude_change: Variation d'altitude")
print("   - velocity_change: Variation de vitesse")
print("   - observation_rank: Rang de l'observation")

print("\n🔍 Aperçu des features temporelles :")
df_temporal.select(
    "icao24", "event_timestamp", "altitude_meters", "prev_altitude", 
    "altitude_change", "velocity_kmh", "velocity_change", "observation_rank"
).orderBy("icao24", "event_timestamp").show(10, truncate=False)

In [ ]:
# Cellule 5 : Features avec fenêtre glissante (Rolling Window)

from pyspark.sql.window import Window
from pyspark.sql.functions import avg, stddev

print("📊 Création des features avec fenêtre glissante (rolling window)...")

# Fenêtre des 5 dernières observations par avion
rolling_window = Window.partitionBy("icao24").orderBy("event_timestamp").rowsBetween(-5, 0)

# Créer les features rolling
df_rolling = df_temporal \
    .withColumn("rolling_avg_altitude", avg("altitude_meters").over(rolling_window)) \
    .withColumn("rolling_std_altitude", stddev("altitude_meters").over(rolling_window)) \
    .withColumn("rolling_avg_velocity", avg("velocity_kmh").over(rolling_window))

print("✅ Features rolling créées :")
print("   - rolling_avg_altitude: Moyenne altitude sur 5 observations")
print("   - rolling_std_altitude: Écart-type altitude sur 5 observations")
print("   - rolling_avg_velocity: Moyenne vitesse sur 5 observations")

print("\n🔍 Aperçu des features rolling :")
df_rolling.select(
    "icao24", "event_timestamp", "altitude_meters", "rolling_avg_altitude", 
    "rolling_std_altitude", "velocity_kmh", "rolling_avg_velocity"
).orderBy("icao24", "event_timestamp").show(10, truncate=False)

In [ ]:
# Cellule 6 : Création du label flight_phase

from pyspark.sql.functions import when, col

print("🏷️  Création du label de classification 'flight_phase'...")

# Créer le label basé sur des règles métier
df_ml = df_rolling.withColumn(
    "flight_phase",
    when(col("on_ground") == True, "GROUND")
    .when((col("altitude_change") > 50) & (col("altitude_meters") < 3000), "TAKEOFF")
    .when(col("altitude_change") > 20, "CLIMB")
    .when(col("altitude_change").between(-20, 20) & (col("altitude_meters") > 8000), "CRUISE")
    .when(col("altitude_change") < -20, "DESCENT")
    .otherwise("TRANSITION")
)

print("✅ Label 'flight_phase' créé avec les catégories :")
print("   - GROUND: Avion au sol")
print("   - TAKEOFF: Décollage (altitude < 3000m, montée rapide)")
print("   - CLIMB: Montée (altitude change > 20m)")
print("   - CRUISE: Croisière (altitude stable > 8000m)")
print("   - DESCENT: Descente (altitude change < -20m)")
print("   - TRANSITION: Autre")

print("\n📊 Distribution des phases de vol :")
df_ml.groupBy("flight_phase").count().orderBy("count", ascending=False).show()

In [ ]:
# Cellule 7 : Sauvegarder en Delta (Silver_ML)

print(f"💾 Sauvegarde des données ML dans {SILVER_ML_PATH}...")

df_ml.write.format("delta").mode("overwrite").save(SILVER_ML_PATH)

print(f"✅ Données ML sauvegardées dans {SILVER_ML_PATH}")
print(f"📦 Nombre de lignes sauvegardées : {df_ml.count():,}")

In [ ]:
# Cellule 8 : Vérification

print("🔍 Vérification des données Silver_ML...")

# Relire les données pour vérifier
df_verify = spark.read.format("delta").load(SILVER_ML_PATH)

# Afficher le schéma final
print("\n📋 Schéma final des données ML :")
df_verify.printSchema()

# Compter les lignes par flight_phase
print("\n📊 Distribution des phases de vol :")
phase_distribution = df_verify.groupBy("flight_phase").count().orderBy("count", ascending=False)
phase_distribution.show()

# Statistiques par phase
print("\n📈 Statistiques par phase de vol :")
df_verify.groupBy("flight_phase").agg(
    avg("altitude_meters").alias("avg_altitude"),
    avg("velocity_kmh").alias("avg_velocity"),
    avg("altitude_change").alias("avg_altitude_change")
).orderBy("flight_phase").show()

# Afficher quelques exemples
print("\n🔍 Exemples de données finales :")
df_verify.select(
    "icao24", "event_timestamp", "altitude_meters", "velocity_kmh",
    "altitude_change", "rolling_avg_altitude", "flight_phase"
).show(10, truncate=False)

print("\n✅ Vérification terminée ! Les données sont prêtes pour le Machine Learning.")